In [1]:
from stdmodandoption import *
import time

runtodo='m12imhdcvhr'
i=580
info=SSF.outdirname(runtodo, i)
rundir=info['rundir']
Nsnapstring=info['Nsnapstring']
havecr=info['havecr']
haveB=info['haveB']
cutcold=0
dx=dy=dz=1
commonpath='/home/tkc004/scratch/snipshot/philruns/'
#SSF.mkdir_p(commonpath+rundir+'/output/withinr200spno100')
fname=commonpath+rundir+'/output/withinr200spno100/snipshot_'+Nsnapstring+'.hdf5'
data = RSS.readsnipshot(fname,ptypelist = [0,1,4])

G = data[0]; DM = data[1]; S = data[2];

rundir /m12i_mass7000_MHDCR_tkFIX/mhdcv/


In [2]:

def calgfromparlocal(G,pos,withinr,spno):
    data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
    gxl = data['gx']; gyl = data['gy']; gzl = data['gz'];
    return [gxl,gyl,gzl]

def calgfromparlocal1(G,pos,withinr,spno,i):
    data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
    gxl = data['gx']; gyl = data['gy']; gzl = data['gz'];
    return [[i,gxl,gyl,gzl]]

def calgfromparlocal3(Glist,pos,withinr,spno,i):
    gxl = np.zeros(len(pos)); gyl = np.zeros(len(pos)); gzl = np.zeros(len(pos))
    for G in Glist:
        data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
        gxl += data['gx']; gyl += data['gy']; gzl += data['gz'];
    data = {}
    data['gx']=gxl; data['gy']=gyl; data['gz']=gzl;
    return [[i,data]]

def pressureXYZlocal(G, pos, dx, dy, dz,havecr,haveB,cutcold):
    data = CRTF.pressureXYZ(G, pos, dx, dy, dz,havecr=havecr,haveB=haveB,cutcold=cutcold)
    return [[i,data]]

def calgfromparlocal2(G,withinr,spno,pos):
    data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
    gxl = data['gx']; gyl = data['gy']; gzl = data['gz'];
    return [gxl,gyl,gzl]

def chunks(l, n):
    n = max(1, n)
    lenl = len(l)
    chucklist = [l[i:i+n] for i in xrange(0, lenl, n)]
    return chucklist
        
def collect_results(result):
    results.extend(result)
    
def joindata(results):
    argsortlist=np.argsort([results[i][0] for i in range(len(results))])
    totdict = np.array([results[i][1] for i in range(len(results))])
    totdict=totdict[argsortlist]
    comdict={}
    for key in totdict[0]:
        totarray=np.array([])
        for indict in totdict:
            totarray = np.append(totarray,indict[key])
        comdict[key] = totarray.flatten()
    return comdict

In [12]:
maxlength=20
nogrid=10

zmax=maxlength/2.0
xlist = ylist = zlist = np.linspace(-zmax,zmax,num=nogrid)
xl,yl,zl = np.meshgrid(xlist,ylist,zlist)
xl = np.ravel(xl); yl = np.ravel(yl); zl = np.ravel(zl);
gxl=np.array([]); gyl=np.array([]); gzl=np.array([]);

In [13]:

start = time.time()
for x,y,z in zip(xl,yl,zl):
    Gdata = SSF.calgfrompar(G,pos=[x,y,z],withinr=200,spno=1) 
    Ggx = Gdata['gx']; Ggy = Gdata['gy']; Ggz = Gdata['gz']
    Sdata = SSF.calgfrompar(S,pos=[x,y,z],withinr=200,spno=1) 
    Sgx = Sdata['gx']; Sgy = Sdata['gy']; Sgz = Sdata['gz']
    DMdata = SSF.calgfrompar(DM,pos=[x,y,z],withinr=200,spno=1) 
    DMgx = DMdata['gx']; DMgy = DMdata['gy'];DMgz = DMdata['gz']
    gx = Ggx+Sgx+DMgx
    gy = Ggy+Sgy+DMgy
    gz = Ggz+Sgz+DMgz
    gxl=np.append(gxl,gx); gyl=np.append(gyl,gy); gzl=np.append(gzl,gz);
end = time.time()
print(end - start)


20.5593070984


In [4]:
pos=[]
for x,y,z in zip(xl,yl,zl):
    pos.append([x,y,z])

nocpu = 20

lenpos = len(pos)

chunksize = lenpos/nocpu

listpos = chunks(pos, chunksize)

In [5]:
import multiprocessing as mp

start = time.time()


withinr=200; spno=1
# Step 1: Init multiprocessing.Pool()


pool = mp.Pool(nocpu)
results=[]

# Step 2: `pool.apply` the `howmany_within_range()
gxyz = [pool.apply_async(calgfromparlocal3, args=([G,S,DM],xyz,withinr,spno,i), callback=collect_results)  for i, xyz in enumerate(listpos)]

# Step 3: Don't forget to close
pool.close()
pool.join()

end = time.time()
print(end - start)

4.86760878563


In [225]:
argsortlist=np.argsort([results[i][0] for i in range(len(results))])
gxlist =np.array([results[i][1] for i in range(len(results))])
gxlnew = gxlist[argsortlist].flatten()
gylist =np.array([results[i][2] for i in range(len(results))])
gylnew = gylist[argsortlist].flatten()
gzlist =np.array([results[i][3] for i in range(len(results))])
gzlnew = gzlist[argsortlist].flatten()

In [10]:
comdict=joindata(results)
gxlnew = comdict['gx']; gylnew = comdict['gy']; 

In [176]:
gxl

array([-4.84727319e-09, -5.12538154e-09, -5.35868326e-09, ...,
        5.51469540e-09,  5.24285733e-09,  5.05639773e-09])

In [177]:
gyl

array([-5.02323516e-09, -5.30243032e-09, -5.66282125e-09, ...,
        5.30635004e-09,  5.07255715e-09,  4.85272221e-09])

In [182]:
gzl

array([-5.56743416e-09, -5.53273332e-09, -5.43101911e-09, ...,
        6.16164624e-09,  6.32919555e-09,  6.26016271e-09])

In [198]:
sortlist

array([ 0,  8, 12,  1,  9,  2, 15, 13, 11, 17, 18, 16,  6,  5, 10,  3, 19,
        4, 14,  7])

In [14]:
gylnew-gyl

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.